In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("fivethirtyeight")

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, GRU, Bidirectional
from keras.optimizers import SGD

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import math

import warnings
warnings.filterwarnings("ignore")

This is a function to plot test and predicted values

In [3]:
def plot_predictions(test, predicted):
    plt.plot(test, color="red", label="real IBM stock price")
    plt.plot(predicted, color="blue", label="predicted stock price")
    plt.title("IBM stock price prediction")
    plt.xlabel("time")
    plt.ylabel("IBM stock price")
    plt.legend()
    plt.show()

Function to calculate RMSE

In [2]:
def return_rmse(test, predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    print("the root mean squared error is : {}.".format(rmse))

In [4]:
data = pd.read_csv("../../data/Stock Regression/TSLA Historical Data 2015-2024.csv", index_col='Date', parse_dates=["Date"])

data.shape

(2486, 6)

In [6]:
data.head(5)

,Price,Open,High,Low,Vol.(Millions),Change
Date,,,,,,
11/15/2024,320.72,310.57,324.68,309.22,114.44,0.0307
11/14/2024,311.18,327.69,329.98,310.37,120.73,-0.0577
11/13/2024,330.24,335.85,344.60,322.50,125.41,0.0053
11-12-2024,328.49,342.74,345.84,323.31,155.73,-0.0615
11-11-2024,350.00,346.30,358.64,336.00,210.52,0.0896
